# [Module 6.1] Train a Model using SageMaker Component in Kubeflow

<font color="red">**만일 이 노트북을 실행하기 전에 [관련 가이드](install_EKS_Kubeflow/README.md) 안보셨다면, 먼저 보시고 가이드를 따라 가세요.**</font>

이 노트북은 Kubeflow 노트북 서버에서 실행이 됩니다. 아래와 같은 작업을 실행 합니다.

- 필요한 Package를 설치 합니다. (AWS boto3, Kubeflow Pipeline SDK)
- SageMaker Components를 가져옵니다.
- S3의 입력 데이타를 설정 합니다.
- Kubeflow Pipeline을 정의 합니다.
- Kubeflow Experiment를 실행 합니다.

**아래 일부 코드는 하드코드가 있습니다. (예: S3 데이타 경로, Region 이름). 이는 EKS/Kubeflow 설치환경 및 데이타 장소에 종속적입니다.실제 환경 구성후에 실행시 변경 해야 합니다.**

---
이 노트북의 실행 시간은 **약 15분** 걸립니다. <br>
2개의 ml.p3.2xlarge instance type으로 학습시에 약 15분 소요 됩니다.


아래는 이 노트북이 Kubeflow 노트북에서 실행이 되어 완료 하면 Kubeflow Dashboard에 나오는 화면 입니다.

![kubeflowPipeline](img/kubeflow-pipeline.png)

아래는 Kubeflow pipeline에서 SageMaker training job 이 실행되고 있는 화면 입니다.

![kubeflow-training](img/kubeflow-training.png)

아래는 Kubeflow pipeline에서 SageMaker creating model job 이 실행되고 있는 화면 입니다.

![kubeflow-creating-model](img/kubeflow-creating-model.png)

아래는 SageMaker Console에 가서 training job 이 실행된 것을 확인 합니다.

![kubeflow-sagemaker-train-job](img/kubeflow-sagemaker-train-job.png)

# AWS boto3 package 설치

**아래 pip install boto3 가 에러시 커널을 리스트하고 해주세요**

In [ ]:
! pip install boto3 --user

## Install Kubeflow Pipelines SDK

In [ ]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade --user

**Resion이 ap-northeast-2 가 아니면 해당 Region으로 변경 해주세요**

In [ ]:
import boto3

#################################
#################################
# REPLACE AWS_REGION= with the current region
#  surround with single quotes
AWS_REGION='ap-northeast-2'

AWS_ACCOUNT_ID=boto3.client('sts').get_caller_identity().get('Account')
print('Account ID: {}'.format(AWS_ACCOUNT_ID))

S3_BUCKET='sagemaker-{}-{}'.format(AWS_REGION, AWS_ACCOUNT_ID)
print('S3 Bucket: {}'.format(S3_BUCKET))

# Build Pipeline 

## 1. Run the following command to load Kubeflow Pipelines SDK

In [ ]:
import kfp
from kfp import components
from kfp import dsl
from kfp.aws import use_aws_secret

## 2.Load reusable sagemaker components

In [ ]:
# 아래는 과거 버전의 sagemaker_train_op 임
# sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/train/component.yaml')
sagemaker_train_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/cb36f87b727df0578f4c1e3fe9c24a30bb59e5a2/components/aws/sagemaker/train/component.yaml')
sagemaker_model_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/model/component.yaml')
sagemaker_deploy_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/0ad6c28d32e2e790e6a129b7eb1de8ec59c1d45f/components/aws/sagemaker/deploy/component.yaml')


**아래의 S3 train, validation, test 의 경로는 하드코딩이 되어 있습니다.** <br>
SageMaker Notebook으로 되돌아 가셔서 <br>
<font color="red">"1.1.Prepare-Tweet-Data.ipynb, 2.1.Convert-Input-TFRecord.ipynb" 노트묵을 실행하고 아래의 변수들의 값을 프린트하여 그 값을 아래의 변수에 할당 하세요</font>
```
print(processed_train_data_s3_uri)
print(processed_validation_data_s3_uri)
print(processed_test_data_s3_uri)
```

## 입력 데이타 정의
**[중요] 아래의 각 경로를 수정 해야합니다.**

In [ ]:
s3_train = "<print(processed_train_data_s3_uri)>"
print("s3_train: \n", s3_train)


s3_validation = "print(processed_validation_data_s3_uri)>"
print("s3_validation: \n", s3_validation)


s3_test = "<print(processed_test_data_s3_uri)>"
print("s3_test: \n", s3_test)

아래와 같이 입력 채널을 정의 합니다. 

In [ ]:
channels='[ \
                    { \
                        "ChannelName": "train", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "'+s3_train+'", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    }, \
                    { \
                        "ChannelName": "validation", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "'+s3_validation+'", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    }, \
                    { \
                        "ChannelName": "test", \
                        "DataSource": { \
                            "S3DataSource": { \
                                "S3DataType": "S3Prefix", \
                                "S3Uri": "'+s3_test+'", \
                                "S3DataDistributionType": "ShardedByS3Key" \
                            } \
                        }, \
                        "CompressionType": "None", \
                        "RecordWrapperType": "None" \
                    } \
                ]'

### Parameter 정의

In [ ]:
epochs= "2"
train_steps_per_epoch= "10"

max_seq_length = "32"
learning_rate= "1e-5"
epsilon= "0.00000001"
train_batch_size= "128"
validation_batch_size= "128"
test_batch_size= "128"

validation_steps= "100"
test_steps= "100"

train_instance_count= "2" 
train_instance_type='ml.p3.2xlarge'
train_volume_size= "1024"

use_xla= "True"
use_amp= "True"
freeze_bert_layer= "True"
enable_checkpointing= "True"
input_mode='Pipe'

## 3.Create Pipeline

<font color="red">**SageMaker의 Train 을 실행할 Role 을 적어주세요.**</font>
각자 구성한 역할의 ARN을 넣으셔야 합니다.

In [ ]:
# SAGEMAKER_ROLE_ARN = 'arn:aws:iam::343441690612:role/service-role/AmazonSageMaker-ExecutionRole-20200801T163342'
SAGEMAKER_ROLE_ARN = "<>"

"4.2.1.Make-Custom-Inference-Image-ECR.ipynb" 에서 생성하여 ECR에 등록한 Image의 ARN을 아래에 넣어 주세요

<font color="red">아래를 각자의 환경에 맞게 수정하셔야 합니다.</font>

아래 예시 처럼 ECR 에 가셔서 본인의 이미지 확인 하세요.

![ECRImages.png](img/ECRImages.png)

In [ ]:
# AWS_ECR_TRAIN_REGISTRY = "343441690612.dkr.ecr.ap-northeast-2.amazonaws.com/bert2tweet:latest"
AWS_ECR_TRAIN_REGISTRY = "<>"

# Inference Image로 아래 image를 사용하여 SageMaker Model Object 생성    
# TF_INFER_IMAGE = '520713654638.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:1.14.0-gpu'
TF_INFER_IMAGE = '<>'

model_output_prefix = 'sagemaker-scikit-learn-2020-08-02-01-14-52-546/model'
model_output_path = 's3://{}/{}'.format(S3_BUCKET,model_output_prefix )
# model_output_path = 's3://sagemaker-us-west-2-057716757052/sagemaker-scikit-learn-2020-06-28-05-08-39-660/model'

In [ ]:
@dsl.pipeline(
    name='Tweet BERT Classification pipeline',
    description='Tweet BERT Classification using KMEANS in SageMaker'
)
def tweet_BERT(
    region = AWS_REGION,
    image = AWS_ECR_TRAIN_REGISTRY,
    dataset_path = channels,
    instance_type = 'ml.p3.8xlarge',
    instance_count = 2,
    volume_size = '50',
    model_putput_path = model_output_path,
    role_arn = SAGEMAKER_ROLE_ARN,
    network_isolation='False',
    traffic_encryption='False',
    spot_instance='False'    
    ):
    # Component 1
    training = sagemaker_train_op(
        region = region,
        image = image,
        channels=channels,        
        instance_type = instance_type,
        instance_count = instance_count,
        volume_size = volume_size,
        model_artifact_path=model_output_path,
        role=role_arn,
        network_isolation=network_isolation,
        traffic_encryption=traffic_encryption,
        spot_instance=spot_instance,        
        hyperparameters={'epochs': epochs,
                        'learning_rate': learning_rate,
                        'epsilon': epsilon,
                        'train_batch_size': train_batch_size,
                        'validation_batch_size': validation_batch_size,
                        'test_batch_size': test_batch_size,                                             
                        'train_steps_per_epoch': train_steps_per_epoch,
                        'validation_steps': validation_steps,
                        'test_steps': test_steps,
                        'use_xla': use_xla,
                        'use_amp': use_amp,                                             
                        'max_seq_length': max_seq_length,
                        'freeze_bert_layer': freeze_bert_layer,
                        'enable_checkpointing': enable_checkpointing
                        },        
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))        
    # Component 2
    create_model = sagemaker_model_op(
        region = region,
        image = TF_INFER_IMAGE,
        model_artifact_url = training.outputs['model_artifact_url'],
        model_name = training.outputs['job_name'],
        role = role_arn
    ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))
    
#     # Component 3
#     prediction = sagemaker_deploy_op(
#         region=region,
#         model_name=create_model.output
#     ).apply(use_aws_secret('aws-secret', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY'))


Compile 하여 tweet_BERT 함수를 tweet_BERT.zip 으로 만듦니다.

In [ ]:
kfp.compiler.Compiler().compile(tweet_BERT, 'tweet_BERT.zip')

아래의 압축을 해제하면 아래와 같은 pipeline 정의를 가지고 있는 yaml 파일이 생성 됩니다.
```
Archive:  ./tweet_BERT.zip
  inflating: pipeline.yaml 
```

In [ ]:
!unzip -o ./tweet_BERT.zip

In [ ]:
# !cat pipeline.yaml

In [ ]:
import time

## 4.Create Kubeflow Experiment

Kubeflow Experiment를 생성하여 실행하면 SageMaker에서 pipeline.yaml 이 실행 됨

In [ ]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')

exp_name    = f'tweet-BERT-train-deploy-kfp-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
my_run = client.run_pipeline(aws_experiment.id, exp_name, 'tweet_BERT.zip')